In [1]:
# handle dataframe
import numpy as np
import pandas as pd

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# imputation
from feature_engine.imputation import(
    AddMissingIndicator,
    MeanMedianImputer,
    CategoricalImputer
)

# preprocessing
from feature_engine.transformation import (
    YeoJohnsonTransformer,
    LogTransformer
)
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import Binarizer

# encoding
from feature_engine.encoding import (
    RareLabelEncoder,
    WoEEncoder
)

import preprocessors as pp

# Load Data

In [2]:
# load data
data = pd.read_csv("loan_data_2007_2014.csv", index_col = [0], dtype = {"desc" : str})

# row x columns
print(data.shape)

# display dataframe
data.head()

(466285, 74)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Formatting Date Data 

In [3]:
# get date columns

date_data = [col for col in data.columns if col.endswith("_d")]
date_data = date_data + ["earliest_cr_line"]
date_data

['issue_d',
 'last_pymnt_d',
 'next_pymnt_d',
 'last_credit_pull_d',
 'earliest_cr_line']

In [4]:
def convert_to_datetime(s):

    if pd.isna(s):
        return s
    
    else:
        # Split the string into month and year
        parts = s.split('-')
        
        if len(parts) == 2:
            # Attempt to convert both parts to integers
            if parts[1].isdigit():
                month, year = parts
            else:
                year, month = parts

            # Determine the century based on the year value
            if int(year) >= 30:
                century = 1900
            else:
                century = 2000

            # Convert to datetime with "01" as the default day
            date_str = f'01-{month}-{str(century + int(year))}'
            return date_str

In [5]:
# convert each date columns with date
for date in date_data:
    data[date] = data[date].apply(convert_to_datetime)
    data[date] = pd.to_datetime(data[date], format = "%d-%b-%Y")

In [6]:
data[date_data]

,issue_d,last_pymnt_d,next_pymnt_d,last_credit_pull_d,earliest_cr_line
0,2011-12-01,2015-01-01,NaT,2016-01-01,1985-01-01
1,2011-12-01,2013-04-01,NaT,2013-09-01,1999-04-01
2,2011-12-01,2014-06-01,NaT,2016-01-01,2001-11-01
3,2011-12-01,2015-01-01,NaT,2015-01-01,1996-02-01
4,2011-12-01,2016-01-01,2016-02-01,2016-01-01,1996-01-01
...,...,...,...,...,...
466280,2014-01-01,2016-01-01,2016-02-01,2016-01-01,2003-04-01
466281,2014-01-01,2014-12-01,NaT,2016-01-01,1997-06-01
466282,2014-01-01,2016-01-01,2016-02-01,2015-12-01,2001-12-01
466283,2014-01-01,2014-12-01,NaT,2015-04-01,2003-02-01


In [7]:
data[date_data].isna().sum()

issue_d                    0
last_pymnt_d             376
next_pymnt_d          227214
last_credit_pull_d        42
earliest_cr_line          29
dtype: int64

# Drop Unused

In [8]:
# replace none to nan
data = data.replace({None: np.nan})

In [9]:
col_to_drop = [
    # all vars na
    'annual_inc_joint'
    , 'dti_joint'
    , 'verification_status_joint'
    , 'open_acc_6m'
    , 'open_il_6m'
    , 'open_il_12m'
    , 'open_il_24m'
    , 'mths_since_rcnt_il'
    , 'total_bal_il'
    , 'il_util'
    , 'open_rv_12m'
    , 'open_rv_24m'
    , 'max_bal_bc'
    , 'all_util'
    , 'inq_fi'
    , 'total_cu_tl'
    , 'inq_last_12m'

    # free text
    , "desc"
    , "emp_title"
    , "title"

    # all unique
    , 'id'
    , 'member_id' 
    , 'url'

    # constant
    , "policy_code"
    , "application_type"

    #
    , "sub_grade"
    , "zip_code"

    # highly dominated by one value
    , "acc_now_delinq"
    , "collections_12_mths_ex_med"
    , "pymnt_plan"
]

In [10]:
# drop unused
data.drop(col_to_drop, axis = 1, inplace = True)

In [11]:
# shape of data after drop
data.shape

(466285, 44)

# Target

In [12]:
# target mapping value
target_map = {
    "Current" : "1"
    , "Fully Paid" : "1"
    , "Charged Off" : "0"
    , "Default" : "0"
    , "Late (31-120 days)" : "0"
    , "Does not meet the credit policy. Status:Charged Off" : "0"
}

data = data[data["loan_status"].isin([
    "Current"
    , "Fully Paid"
    , "Charged Off"
    , "Default"
    , "Late (31-120 days)"
    , "Does not meet the credit policy. Status:Charged Off"
])]

# change value
data["loan_status"] = data["loan_status"].map(target_map)

# Split to train and test

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop("loan_status", axis = 1),
    data[["loan_status"]],
    test_size = 0.2,
    stratify = data["loan_status"],
    random_state = 0
)

X_train.shape, X_test.shape

((367946, 43), (91987, 43))

In [14]:
y_train

,loan_status
215242,1
287490,1
46837,1
402060,1
169786,1
...,...
116534,1
49987,1
303127,1
346701,1


# Config

In [15]:
# ---NUMERICAL VARIABLES TO IMPUTE---

# Numerical vars with na mean
NUMERICAL_VARS_WITH_NA_MEAN = [
    "inq_last_6mths"
    , "open_acc"
    , "total_acc"
    , "mths_since_last_delinq"
    , "mths_since_last_major_derog"
    , "revol_util"
    , "mths_since_last_record"
]

# Numerical vars witth median
NUMERICAL_VARS_WITH_NA_MEDIAN =[
    "tot_cur_bal"
    , "total_rev_hi_lim"
    , "delinq_2yrs"
    , "pub_rec"
    , "tot_coll_amt"
]

# ---NUMERICAL VARIABLES TRANSFORMATION---

NUMERICAL_YEO_VARS = [
    "tot_cur_bal"
    , "total_rev_hi_lim"
    , "revol_bal"
    , "total_rec_int"
]

NUMERICAL_LOG_VARS = [
    "annual_inc"
]

BINARIZE_VARS = [
    "delinq_2yrs"
    , "pub_rec"
    , "tot_coll_amt"
    , "collection_recovery_fee"
    , "recoveries"
    , "total_rec_late_fee"
]

# ---CATEGORICAL VARIABLES TO IMPUTE---

CATEGORICAL_DATE_WITH_NA_CONST = [
    "next_pymnt_d"
    , "last_pymnt_d"
    , "last_credit_pull_d"
    , "earliest_cr_line"
]

CATEGORICAL_DATE_DIFFERENCE = [
    "issue_d"
    , "next_pymnt_d"
    , "last_pymnt_d"
    , "last_credit_pull_d"
    , "earliest_cr_line"
]

CATEGORICAL_VARS_WITH_NA_FREQUENT = [
    "emp_length"
]

CATEGORICAL_RARE = [
    "home_ownership"
    , "grade"
    , "purpose"
    , "addr_state"
]

CATEGORICAL_WOE = [
    "initial_list_status"
    , "term"
    , "verification_status"
    , "home_ownership"
    , "grade"
    , "emp_length"
    , "purpose"
    , "addr_state"
]



# Pipeline

In [16]:
pipe = Pipeline([
    # Add missing indicator
    ("mean_missing_indicator", AddMissingIndicator(variables = NUMERICAL_VARS_WITH_NA_MEAN)),
    ("median_missing_indicator", AddMissingIndicator(variables = NUMERICAL_VARS_WITH_NA_MEDIAN)),

    # Imputation for numerical
    ("mean_imputer", MeanMedianImputer(imputation_method = "mean", variables = NUMERICAL_VARS_WITH_NA_MEAN)),
    ("median_imputer", MeanMedianImputer(imputation_method = "median", variables = NUMERICAL_VARS_WITH_NA_MEDIAN)),

    # Imputation for categorical frequent
    ("frequent_imputer", CategoricalImputer(imputation_method = "frequent", variables = CATEGORICAL_VARS_WITH_NA_FREQUENT)),
    ("const_date_imputer", pp.ImputeConstDatetime(value = "01-Dec-2017", variables = CATEGORICAL_DATE_WITH_NA_CONST)),
    
    # Numerical Transformation
    ("YeoJohnson_transform", YeoJohnsonTransformer(variables = NUMERICAL_YEO_VARS)),
    ("log_transform", LogTransformer(variables = NUMERICAL_LOG_VARS)),

    # Binning
    ("binning", SklearnTransformerWrapper(transformer = Binarizer(threshold = 0.1), variables = BINARIZE_VARS)),

    # Rare
    ("rare", RareLabelEncoder(tol = 0.01, n_categories = 2, variables = CATEGORICAL_RARE)),

    # WOE
    ("woe", WoEEncoder(variables = CATEGORICAL_WOE)),

    # date difference
    ("date_difference", pp.TimeDifference(variables = CATEGORICAL_DATE_DIFFERENCE, reference_date = "01-Dec-2017"))
])


In [17]:
pipe.fit(X_train, y_train)

c:\Users\Annur Afgoni\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('mean_missing_indicator',
                 AddMissingIndicator(variables=['inq_last_6mths', 'open_acc',
                                                'total_acc',
                                                'mths_since_last_delinq',
                                                'mths_since_last_major_derog',
                                                'revol_util',
                                                'mths_since_last_record'])),
                ('median_missing_indicator',
                 AddMissingIndicator(variables=['tot_cur_bal',
                                                'total_rev_hi_lim',
                                                'delinq_2yrs', 'pub_rec',
                                                'tot_coll_amt'])),
                ('mean_...
                                  variables=['home_ownership', 'grade',
                                             'purpose', 'addr_state'])),
                ('woe',
                 WoEEncoder(variables=['initial_list_status', 'term',
                                       'verification_status', 'home_ownership',
                                       'grade', 'emp_length', 'purpose',
                                       'addr_state'])),
                ('date_difference',
                 TimeDifference(reference_date='01-Dec-2017',
                                variables=['issue_d', 'next_pymnt_d',
                                           'last_pymnt_d', 'last_credit_pull_d',
                                           'earliest_cr_line']))])

In [18]:
X_train = pipe.transform(X_train)
X_test = pipe.transform(X_test)

In [19]:
# Save

X_train.to_csv("x_train.csv", index = False, header = True)
X_test.to_csv("x_test.csv", index = False, header = True)

y_train.to_csv("y_train.csv", index = False, header = True)
y_test.to_csv("y_test.csv", index = False, header = True)

In [20]:
X_train[date_data]

,issue_d,last_pymnt_d,next_pymnt_d,last_credit_pull_d,earliest_cr_line
215242,2009,975,0,700,8796
287490,1157,700,669,700,6940
46837,1461,731,669,700,6088
402060,1340,1006,0,700,9345
169786,1764,1675,0,914,7731
...,...,...,...,...,...
116534,1614,700,669,700,9497
49987,1461,700,669,700,5754
303127,1187,700,669,700,7884
346701,1249,700,669,700,6209


In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 367946 entries, 215242 to 163622
Data columns (total 55 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   loan_amnt                       367946 non-null  int64  
 1   funded_amnt                     367946 non-null  int64  
 2   funded_amnt_inv                 367946 non-null  float64
 3   term                            367946 non-null  float64
 4   int_rate                        367946 non-null  float64
 5   installment                     367946 non-null  float64
 6   grade                           367946 non-null  float64
 7   emp_length                      367946 non-null  float64
 8   home_ownership                  367946 non-null  float64
 9   annual_inc                      367946 non-null  float64
 10  verification_status             367946 non-null  float64
 11  issue_d                         367946 non-null  int64  
 12  purpose    

In [24]:
y_train["loan_status"].unique()

array(['1', '0'], dtype=object)

In [25]:
y_test["loan_status"].unique()

array(['1', '0'], dtype=object)